# Learning Fair Representations, python implementation

# Install and load required packages

In [1]:
import numpy as np
import scipy.optimize as scipyopt
import pandas as pd
from numba import jit

import functools

In [2]:
# Timing
# http://pythoncentral.io/time-a-python-function/

import timeit
# timeit.timeit('"-".join(str(n) for n in range(100))', number=10000)
# timeit.Timer('for i in range(10): oct(i)', 'gc.enable()').timeit() # with gc

def wrapper(func, *args, **kwargs):
    def wrapped():
        return func(*args, **kwargs)
    return wrapped

# >>> def costly_func(lst): 
# ...     return map(lambda x: x^2, lst) 
# ... 
# >>> short_list = range(10) 
# >>> wrapped = wrapper(costly_func, short_list)
# >>> timeit.timeit(wrapped, number=1000)

In [3]:
def tst(a, b):
    return a + b

wrp_tst = functools.partial(tst, 1)
wrp_tst(2)

3

# Load the "Adult" dataset
The Adult dataset is from [here](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/).
In categorical data, missing data is handled as just another category. This data set does not contain NA-values in the continuous features.

In [4]:
filename_orig_data = "data/adult.data"
filename_orig_test = "data/adult.test"
# NOTE: if the data set is very large, we should save the preprocessed data to avoid preprocessing cost.
#filename_data = "data/processed.adult.data"
#filename_test = "data/processed.adult.test"

# missing_data_marker = "?" # can be a string

# df_orig_data = pd.read_csv(filename_orig_data, sep=',',header=None)
# df_orig_data = pd.read_csv(filename_orig_data, sep=',', na_values=['?'])
df_orig_data = pd.read_csv(filename_orig_data, sep=',', na_values=['?'], skipinitialspace=True)
df_orig_test = pd.read_csv(filename_orig_test, sep=',', na_values=['?'], skipinitialspace=True)
# print(df_orig_data)
print(df_orig_data.keys())

# df_orig_data = readtable(filename_orig_data);
# df_orig_test = readtable(filename_orig_test);

# Not in use, let NA values go through as "?", handled as yet another category
#df_orig_test = readtable(filename_orig_test, nastrings = ["", "NA", "?"]);

# df_orig_test[ 1:3, :age ] # Example: selecting subset of rows (1 to 3), from a certain column

# names(df_orig_data)

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'classification'],
      dtype='object')


## Parameters for our model
Set $K$ to the number of prototypes. The paper does not discuss how this should be chosen. Here we use the dimension of the original data, when sensitive and classification features are removed.

In [5]:
# Columns to encode with OneHot encoding aka Dummy variables
# Remember to not encode sensitive_column_name and classification_column_name
# Has to be an array
columns_to_encode = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']

# Sensitive column is "gender" as in the paper
# NOTE: This column must be have only two possible values (be a binary variable)
#       as the method in the paper assumes this ("protected or not protected").
sensitive_column_name = 'sex'

# NOTE: This column must be have only two possible values (be a binary variable)
#       as the method in the paper assumes this (binary classification).
classification_column_name = 'classification'

# Size or data matrix. -1 for classification/target column and -1 for sensitive column
K = df_orig_data.shape[1] - 2

## Use OneHot encoding (dummy variables) for categorical features

In [6]:
# Vertically concatenate to get the whole dataset
df_orig_all = pd.concat([df_orig_data, df_orig_test])
print("Original data ", df_orig_data.shape, df_orig_test.shape, df_orig_all.shape, "\n")
# Do the One-Hot-Encoding / Dummy variables conversion.
# Missing values in categorical values are encoded simply as yet another category.
# drop_first drops first column resulting from one-hot-encoding, so that the variables are not linearly dependent 
# df_all = pd.get_dummies(df_orig_all, prefix=columns_to_encode, drop_first=True, columns=columns_to_encode)
df_all = pd.get_dummies(df_orig_all, prefix=columns_to_encode, columns=columns_to_encode)

### Map sensitive and classification/target columns to appropriate types.

# Map Sensitive column "Male"/"Female" to true/false.
# Change this if you change the sensitive column
# (maybe do something that automatically just picks one category to be true and other to be false)
df_all[sensitive_column_name] = df_all[sensitive_column_name].map(lambda gender: gender == 'Female')
# Map classification column values to 0 and 1
df_all[classification_column_name] = df_all[classification_column_name].map(lambda label: 1 if label == '>50K' else 0)

# Read out the converted data back to data and test sets.
len_data = df_orig_data.shape[0]
len_test = df_orig_test.shape[0]
df_data = df_all[0:len_data]
df_test = df_all[len_data+1:len_data+len_test]
print("Encoded data ", df_data.shape, df_test.shape, df_all.shape, "\n")
# Julia:
# Original data (32561,15)(16281,15)(48842,15)
# Encoded data (32561,101)(16281,101)(48842,101)
# Python: 
# Original data  (32561, 15) (16281, 15) (48842, 15)
# Encoded data  (32560, 98) (16280, 98) (48842, 98)
# TODO: investigate discrepency

Original data  (32561, 15) (16281, 15) (48842, 15) 

Encoded data  (32561, 105) (16280, 105) (48842, 105) 



# Code for the model

## Notation/Definitions in paper

- $X$ denotes the entire data set of individuals. Each $x \in X$ is a vector of length $D$ where each component of the vector describes some attribute of the person.
- $S$ is a binary random variable representing whether or not a given individual is a member of the protected set; we assume the system has access to this attribute.
- $X_0$ denotes the training set of individuals.
- $X^+ \subset X$, $X_0^+ \subset X_0$ denotes the subset of individuals (from the whole set and the training set respectively) that are members of the protected set (i.e., $S = 1$), and $X^−$ and $X_0^−$ denotes the subsets that are not members of the protected set, i.e., $S = 0$.
- $Z$ is a multinomial random variable, where each of the $K$ values represents one of the intermediate set of ”prototypes”. Associated with each prototype is a vector $\mathbf{v}_k$ in the same space as the individuals $\mathbf{x}$.
- $Y$ is the binary random variable representing the classification decision for an individual, and $f : X \rightarrow Y$ is the desired classification function.
- $d$ is a distance measure on $X$, e.g., simple Euclidean distance: $d(\mathbf{x}_n , \mathbf{v}_k ) = \Vert\mathbf{x}_n − \mathbf{v}_k \Vert_2$.

## Our changes and clarifications

We will differ a bit from the definitions in the paper. The definitions we use are:
- $\mathbf{X}$ denotes the entire data set, a $(N \times D)$ matrix. The rows of the matrix are the feature vectors $\mathbf{x}_n$ representing attributes of an individual. $\mathbf{X}$ contains neither the classification information (target) column, nor the sensitive column.
- $S$ is a binary variable representing whether or not a given individual is a member of the "protected group". For the user of the algorithm, this is a decision that is done before running the algorithm by setting `sensitive_column_name` in the parameters.
- $\mathbf{X}_{train}$ denotes the training set.
- $\mathbf{X}_{test}$ denotes the test set.
- $\mathbf{X}^+$ denotes the subset of individuals that are members of the "protected group" i.e. individuals for whom $S=1$. Similarly $\mathbf{X}^-$ denotes the subset of individuals for whom $S=0$. It's worthwhile to note that for the algorithm it actually doesn't matter if you flip the groups of who is "protected" and who is "non-protected", the result will be the same due to symmetry of statistical parity. So don't get too attached to the terminology.
- Define $\mathbf{X}_{train}^+$, $\mathbf{X}_{train}^-$, $\mathbf{X}_{test}^+$ and $\mathbf{X}_{test}^-$ similarly as above.
- $d$ is a distance measure on $\mathbf{X}$ (e.g. euclidean distance).
- $K$ is the number of prototypes.
- $Z$ is a random integer from the set $\left\{1,\dots,K\right\}$.
- $Y$ is a binary variable representing the classification decision (we consider binary classification only).

Let $Z$ be a random integer from the set $\left\{1,\dots,K\right\}$. Now we can denote the probability that a datapoint $\mathbf{x}$ maps to a particular prototype $k$ with $\mathbb{P}(Z=k \mid \mathbf{x})$ i.e. given a datapoint $\mathbf{x}$, the probability that $Z$, the index of the prototype for that data point, is $k$.

## Definitions in code

From the definitions above, we map some to code and also define additional stuff.

Julia is Column-Majored, so our matrices will be altered accordingly.

- $\mathbf{X}$ is just `𝐗`, but $(D \times N)$ instead of $(N \times D)$.
- $\mathbf{X}_{train}$ is `𝐗train`, $\mathbf{X}_{test}$ is `𝐗test`.
- Grouped versions are `𝐗⁺train`, `𝐗⁻train`, `𝐗⁺test`, and `𝐗⁻test` respectively.
- $S$ is defined as multiple vectors `S_<someset>`, each containing the sensitive column for `<someset>`, e.g. `S_𝐗`.
- $d$ is defined as a lambda function `d` and plain function `de`.
  - The functions implemented here have versions that default to euclidean distance, and versions that accept a user defined distance function.
- $Z$ is replaced by $\mathbf{Z}$, a matrix of probability vectors $\mathbf{z}$.
- The classification information is contained in `𝐲`, `𝐲train`, and `𝐲test`.

Additionally:
- Denote the tuple of prototypes $\mathbf{V} = \left(\mathbf{v}_1,...,\mathbf{v}_K\right)$. Since a single prototype $\mathbf{v}_k$ is a vector of length $D$, $\mathbf{V}$ can be expressed as a ($D \times K$) matrix. This is our optimization variable `𝐕`.
- `A` contains the hyperparameters.

In [7]:
# Sensitive indices for training and test data
S_Xtrain_df = df_data[sensitive_column_name]
S_Xtest_df = df_test[sensitive_column_name]

# Classification vectors for training and test data
ytrain_df = df_data[classification_column_name]
ytest_df = df_test[classification_column_name]

# Drop sensitive and classification columns, set difference
columns_to_drop = set([sensitive_column_name, classification_column_name])
features_left = [x for x in df_data.keys() if x not in columns_to_drop]

# Training and test sets
Xtrain_df = df_data[features_left]
Xtest_df = df_test[features_left]


# Standardize the features to have mean 0 variance 1
# But only standardize non-one-hot-encoded features

# Features that are not one-hot-encoded
non_encoded_features = [x for x in features_left if x.split('_')[0] not in columns_to_encode]
# Find indices of those features
# non_encoded_features = find(symbol -> in(symbol, non_encoded), names(df_data[idxs_left]))

# Calculate mean and var from training set
train_mean_df = Xtrain_df.mean()
train_var_df = Xtrain_df.var()

# Only standardize non-one-hot-encoded features
for ne_feat in non_encoded_features:
    Xtrain_df = Xtrain_df.assign(**{ne_feat: (Xtrain_df.loc[:, ne_feat] - train_mean_df[ne_feat]) / train_var_df[ne_feat]})
    # We need to use the same variance and mean for our test set as in the training set,
    # otherwise they would not be comparable.
    Xtest_df = Xtest_df.assign(**{ne_feat: (Xtest_df.loc[:, ne_feat] - train_mean_df[ne_feat]) / train_var_df[ne_feat]})


# Reconstruct full dataset
X_df = pd.concat([Xtrain_df, Xtest_df])
S_X_df = pd.concat([S_Xtrain_df, S_Xtest_df])
y_df = pd.concat([ytrain_df, ytest_df])

print("Done.")

Done.


### From DataFrames to matrices

In [8]:
### Dimensions
N = X_df.shape[0]
D = X_df.shape[1]
Ntrain = Xtrain_df.shape[0]
Ntest = Xtest_df.shape[0]

### Data matrices
X         = X_df.values
Xtrain    = Xtrain_df.values
Xtest     = Xtest_df.values

S_X       = S_X_df.values
S_Xtrain  = S_Xtrain_df.values
S_Xtest   = S_Xtest_df.values

y         = y_df.values
ytrain    = ytrain_df.values
ytest     = ytest_df.values

### Optimization variables
# Main optimization variable, matrix holding the prototype vectors.
# Initialized to random Float64 matrix, normal distribution 0 mean 1 variance
V = np.random.randn(K, D)
# Weights or "prototype label predictions" (probabilities)
w = np.random.rand(K) # floats in [0,1)
# Alphas
alpha_pos = np.random.rand(D) # 𝛂⁺
alpha_neg = np.random.rand(D) # 𝛂⁻

### Hyperparameters
A = dict = {'z': 1000, 'x': 0.0001, 'y': 0.1};

print("Done.")

Done.


In [9]:
# Check that normalization got us values around 0
Xtrain.max().max(), Xtrain.min().min(), Xtest.max().max(), Xtest.min().min(), V.max(), V.min()

(1.0,
 -1.3719338843612912,
 1.0,
 -1.3719338843612912,
 3.5096356112243225,
 -3.188600743274669)

In [10]:
Xtrain_pos = Xtrain[S_Xtrain]
Xtrain_neg = Xtrain[S_Xtrain==False]
Xtest_pos = Xtest[S_Xtest]
Xtest_neg = Xtest[S_Xtest==False]
Xtrain.shape, Xtrain_pos.shape, Xtrain_neg.shape, S_Xtrain.shape, Xtrain_pos.shape[0]+Xtrain_neg.shape[0]
Xtest.shape, Xtest_pos.shape, Xtest_neg.shape, S_Xtest.shape, Xtest_pos.shape[0]+Xtest_neg.shape[0]
print("Training:", Xtrain_pos.shape, Xtrain_neg.shape, "Test:", Xtest_pos.shape, Xtest_neg.shape)
# Julia: ("Training:",(99,10771),(99,21790),"Test:",(99,5421),(99,10860))

Training: (10771, 103) (21790, 103) Test: (5421, 103) (10859, 103)


Divide training and test sets to groups according to whether the individuals are "protected" or not.

## Mapping $\mathbf{X} \rightarrow \mathbf{Z}$
Now we can define a mapping from the original dataset $\mathbf{X}$ to probabilities via the *softmax function*. [Wikipedia](https://en.wikipedia.org/wiki/Softmax_function):
> Softmax function "squashes" a $K$-dimensional vector $\mathbf{z}$ of arbitrary real values to a $K$-dimensional vector $\sigma(\mathbf{z})$ of real values in the range $[0, 1]$ that add up to 1.

Most notably `softmax` returns a probability vector. We will define a modified version that maps a $D$-dimensional vector $\mathbf{x}$ to a $K$-dimensional vector $\sigma(\mathbf{x})$, i.e. the mapping won't necessarily preserve the dimensionality of $\mathbf{x}$.

Also from [Wikipedia](https://en.wikipedia.org/wiki/Multinomial_logistic_regression):
> $$\operatorname{softmax}(k,x_1,\ldots,x_n) = \frac{e^{x_k}}{\sum_{i=1}^n e^{x_i}}$$
> is referred to as the [*softmax function*](https://en.wikipedia.org/wiki/Softmax_function).  The reason is that the effect of exponentiating the values $x_1,\ldots,x_n$ is to exaggerate the differences between them.  As a result, $\operatorname{softmax}(k,x_1,\ldots,x_n)$ will return a value close to 0 whenever $x_k$ is significantly less than the maximum of all the values, and will return a value close to 1 when applied to the maximum value, unless it is extremely close to the next-largest value.  Thus, the softmax function can be used to construct a weighted average that behaves as a smooth function (which can be conveniently differentiated, etc.) and which approximates the [indicator function](https://en.wikipedia.org/wiki/Indicator_function).

So we define, as in the paper equation (2), $$\mathbb{P}(Z=k \mid \mathbf{x}) = \frac{e^{-d(\mathbf{x}, \mathbf{v}_k)}}{\sum_{j=1}^K e^{-d(\mathbf{x}, \mathbf{v}_j)}}$$
where
- $\mathbb{P}(Z=k \mid \mathbf{x})$ is described in [definitions](#Definitions)
- $\mathbf{x}$ is the datapoint
- $\mathbf{v}_k$ is a vector associated with the $k$th prototype
- $d$ is a distance measure between $\mathbf{x}$ and $\mathbf{v}_k$ (e.g. the euclidean distance)

This means that since we have replaced $x_k$ in the softmax with the negative distance between $\mathbf{x}$ and prototype $\mathbf{v}_k$, the softmax returns a value close to 0 whenever the distance from $\mathbf{x}$ to the prototype $\mathbf{v}_k$ is significantly higher than $\min_{j\in\{1,\dots,K\}}\, d(\mathbf{x}, \mathbf{v}_i)$, and close to 1 when applied to the minimum value.

"Mapping from X to Z" in the paper means mapping the vector $\mathbf{x}$ to a probability vector $\mathbf{z}$ of length $K$ via the softmax function. These probability vectors are then used directly for training the classifier.

**In code** this means that $\mathbb{P}(Z=k \mid \mathbf{x})$ is represented by a function taking
- the data point $\mathbf{x}$ which is a `Vector` of length $D$
- $(D \times K)$ `Matrix` of prototypes $\mathbf{V}$ containing all $K$ prototypes $\mathbf{v}_k$ i.e. each prototype is a `Vector` (remember, Column-Major order on matrices)
- a distance measure on $\mathbf{X}$

and returning
- a `Vector` $\mathbf{z}$ of length $K$ representing a [probability vector](https://en.wikipedia.org/wiki/Probability_vector), where each value $z_k$ of the probability vector $\mathbf{z}$ tells how probable it is that $\mathbf{x}$ maps to $\mathbf{v}_k$. Since $\mathbf{z}$ is a probability vector, $\sum_{i=k}^K z_k = 1$.

We will name this function `softmax` and define it as follows:

In [11]:
V.shape, X.shape

((13, 103), (48841, 103))

Distances are calculated with Eq (12):
$$d(\mathbf{x}_n, \mathbf{v}_k, \alpha) = \sum_{d=1}^D \mathbf{\alpha}_d (\mathbf{x}_{nd}-\mathbf{v}_{kd})^2$$
So the distance matrix is defined by `distances[n, k] = sum(alphas * (X[n,:] - V[k,:])^2)`

In [12]:
np.sum(alpha_pos * np.power(X[1, :] - V[1, :], 2))

55.118001469400888

In [13]:
# Calculate distance matrix
@jit(nopython=True)
def distances(X, V, alpha):
    # X is a (N x D) matrix of datapoints
    # V is a (K x D) matrix of prototypes
    # alpha is a (1 x D) vector of weights
    N = X.shape[0]
    D = X.shape[1]
    K = V.shape[0]
    distances = np.zeros((N, K))
    
    for n in range(N):
        for k in range(K):
            distances[n, k] = np.sum(alpha * np.power(X[n, :] - V[k, :], 2))
    
    return distances


import concurrent.futures
import functools
# Calculate distance matrix
@jit(nopython=True, nogil=True)
def distances_threaded_job(distances, X, V, alpha, N, k):
    for n in range(N):
        distances[n, k] = np.sum(alpha * np.power(X[n, :] - V[k, :], 2))

def distances_threaded(X, V, alpha):
    N = X.shape[0]
    D = X.shape[1]
    K = V.shape[0]
    distances = np.zeros((N, K))
    fun = functools.partial(distances_threaded_job, distances, X, V, alpha, N)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = executor.map(fun, (k for k in range(K)))
    return distances


def distances_threaded_old(X, V, alpha=None):
    # X is a (N x D) matrix of datapoints
    # V is a (K x D) matrix of prototypes
    # alpha is a (1 x D) vector of weights
    N = X.shape[0]
    D = X.shape[1]
    K = V.shape[0]
    distances = np.empty((N, K))
    
    def inside_job(k):
        for n in range(N):
            distances[n, k] = np.sum(alpha * np.power(X[n, :] - V[k, :], 2))
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        fs = {executor.submit(inside_job, k): k for k in range(K)}
        concurrent.futures.wait(fs)
    
    return distances

In [14]:
# # distances(Xtrain, V, alpha_pos)
# wrapped_distances = wrapper(distances, Xtrain, V, alpha_pos)
# timeit.timeit(wrapped_distances, number=5)
# # timeit.Timer(wrapped_distances, 'gc.enable()').timeit(5) # with gc
# # 21.99340252300317, 21.508173273992725
# # numba jit: 7.959764497005381

In [15]:
# wrapped_distances_threaded = wrapper(distances_threaded, Xtrain, V, alpha_pos)
# timeit.timeit(wrapped_distances_threaded, number=10)
# # 3.146800766000524

In [16]:
dists = distances_threaded(Xtrain, V, alpha_pos)

In [17]:
dists.shape

(32561, 13)

In [99]:
# Note: For avoiding numerical infinity problems in softmax, see John D. Cook's advice
# http://www.johndcook.com/blog/2010/01/20/how-to-compute-the-soft-maximum/
# http://cs231n.github.io/linear-classify/#softmax

# Returns (N x K) matrix, a probability vector of length K for each datapoint

@jit(nopython=True,nogil=True)
def softmax_jit_loop(result, distances, N, K):
    for n in range(N):
        for k in range(K):
            # Negation of distance was done already
            result[n,k] = np.exp(distances[n,k])
        # stability
        result[n,:] -= np.max(result[n,:])
        # result
        result[n,:] = result[n,:] * (1/np.sum(result[n,:]))

@jit(nopython=True,nogil=True)
def softmax_jit_loop_stable(result, distances, N):
    for n in range(N):
        # minus np.max for numerical stability
        tmp = np.exp(distances[n,:] - np.max(distances[n,:]))
        # Negation of distance was done already
        result[n,:] = tmp / np.sum(tmp)
        
def softmax(X, V, alpha):
    # Calculate the distances, negate already
    distances = -distances_threaded(X, V, alpha)
    
    N = X.shape[0]
    K = V.shape[0]
    
    result = np.zeros((N, K))
#     softmax_jit_loop(result, distances, N, K)
    softmax_jit_loop_stable(result, distances, N)
        
    return result


@jit(nopython=True,nogil=True)
def softmax_threaded_job_fixed(result, distances, N, k):
    for n in range(N):
        # Negation of distance was done already
        result[n,k] = np.exp(distances[n,k] - np.max(distances[n,:]))

@jit(nopython=True)
def sfix(result, N):
    for n in range(N):
        total = np.sum(result[n,:])
        if total:
            div = total
        else:
            div = 1e-9
        result[n,:] = result[n,:] / div

def softmax_threaded_fixed(X, V, alpha):
    # Calculate the distances, negate already
    distances = -distances_threaded(X, V, alpha)
    
    N = X.shape[0]
    D = X.shape[1]
    K = V.shape[0]
    result = np.empty((N, K))
    
    fun = functools.partial(softmax_threaded_job_fixed, result, distances, N)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = executor.map(fun, (k for k in range(K)))
    
    sfix(result, N)
    return result

# for n in range(N):
#     for k in range(K):
#         # Negation of distance was done already
#         result[n,k] = np.exp(distances[n,k])
#     result[n,:] = result[n,:] * (1/np.sum(result[n,:]))

# @jit(nopython=True,nogil=True)
# def softmax_threaded_joined_job(result, X, V, alpha, N, K):
#     for n in range(N):
#         for k in range(K):
#             # Negation of distance was done already
#             dm = np.sum(alpha * np.power(X[n, :] - V[k, :], 2))
#             result[n,k] = np.exp(-dm)
#         result[n,:] = result[n,:] * (1/np.sum(result[n,:]))

@jit(nopython=True,nogil=True)
def softmax_threaded_joined_job(result, X, V, alpha, N, k):
    for n in range(N):
        # Negation of distance was done already
        dm = -np.sum(alpha * np.power(X[n, :] - V[k, :], 2))
        result[n,k] = np.exp(dm)
        # minus np.max for numerical stability
        tmp = np.exp(distances[n,:] - np.max(distances[n,:]))

def softmax_threaded_joined(X, V, alpha):    
    N = X.shape[0]
    D = X.shape[1]
    K = V.shape[0]
    result = np.empty((N, K))
    
    fun = functools.partial(softmax_threaded_joined_job, result, X, V, alpha, N)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = executor.map(fun, (k for k in range(K)))
    
    sfix(result, N)
    return result

In [101]:
foo1=softmax(Xtrain, V, alpha_pos)
np.sum(foo1)

32561.0

In [92]:
foo2=softmax_threaded_fixed(Xtrain, V, alpha_pos)
np.sum(foo2)

32561.0

In [93]:
foo3=softmax_threaded_joined(Xtrain, V, alpha_pos)
np.sum(foo3)

32561.0

In [40]:
foo1.shape,foo2.shape,foo3.shape

((32561, 13), (32561, 13), (32561, 13))

In [88]:
np.sum(foo1),np.sum(foo2),np.sum(foo3)

(32561.0, 32561.0, 32561.0)

In [98]:
# Verify
foo1.shape[0] == round(np.sum(foo1)), foo2.shape[0] == round(np.sum(foo2)), foo3.shape[0] == round(np.sum(foo3))

(True, True, True)

In [89]:
wrapped_softmax = wrapper(softmax, Xtrain, V, alpha_pos)
timeit.timeit(wrapped_softmax, number=50)
# 14.974193882997497

15.07579768000869

In [50]:
wrapped_softmax_threaded_fixed = wrapper(softmax_threaded_fixed, Xtrain, V, alpha_pos)
timeit.timeit(wrapped_softmax_threaded_fixed, number=50)
# 15=5.419921508990228
# 50=18.265479806999792

15.313272532992414

In [90]:
wrapped_softmax_threaded_joined = wrapper(softmax_threaded_joined, Xtrain, V, alpha_pos)
timeit.timeit(wrapped_softmax_threaded_joined, number=50)
# 15=5.419921508990228
# 50=18.265479806999792

14.90795697500289

In [84]:
np.max(foo1-foo3),np.min(foo1-foo3),np.max(foo1-foo2),np.min(foo1-foo2),np.max(foo2-foo3),np.min(foo2-foo3)

(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [85]:
np.sum(foo1-foo3),np.sum(foo1-foo2),np.sum(foo2-foo3)

(0.0, 0.0, 0.0)

In [86]:
foo1[5,5],foo2[5,5],foo3[5,5]

(2.6691817583990904e-11, 2.6691817583990904e-11, 2.6691817583990904e-11)

In [129]:
# TODO: clean up duplicate code

### FOR VECTORS; give in 𝐱, get 𝐳

# This is same as Eq (2) in paper.
function softmax_dist{T<:Number}(𝐱::Vector{T}, 𝐕::Matrix{T}, distanceMeasure::Function)
    K = size(𝐕, 2)
    res = Vector{Float64}(K)
    denominator = Float64(0.0)
    # Use one loop to calculate both numerator and denominator
    @inbounds for k in 1:K
        res[k] = exp(- distanceMeasure(𝐱, 𝐕[:,k]) )
        denominator += res[k]
    end
    denom = inv(denominator)
    res .* denom
end

function softmax_euclidean{T<:Number}(𝐱::Vector{T}, 𝐕::Matrix{T})
    K = size(𝐕, 2)
    res = Vector{Float64}(K)
    denominator = Float64(0.0)
    # Use one loop to calculate both numerator and denominator
    @inbounds for k in 1:K
        res[k] = exp(- vecnorm(𝐱 - 𝐕[:,k]) )
        denominator += res[k]
    end
    denom = inv(denominator)
    res .* denom
end

### FOR MATRICES; give in 𝐗, get 𝐙

function softmax_dist{T<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{T}, distanceMeasure::Function)
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    # Preallocate result matrix, no need to zero it
    res = Matrix{Float64}(K, N)
    @fastmath @inbounds for n in 1:N
        res[:,n] = softmax_dist(𝐗[:,n], 𝐕, distanceMeasure)
    end
    res
end

# Parallel version
function softmax_dist_par{T<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{T}, distanceMeasure::Function)
    #nprocs()==CPU_CORES || addprocs(CPU_CORES-1)    
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    # Preallocate result matrix, no need to zero it
    res = SharedArray(Float64, (K, N))
    @fastmath @sync @parallel for n in 1:N
        for k in 1:K
            res[k,n] = exp(- distanceMeasure(𝐗[:,n], 𝐕[:,k]) )
        end
        res[:,n] = res[:,n] .* inv(sum(res[:,n]))
    end
    res
end

function softmax_euclidean{T<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{T})
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    # Preallocate result matrix, no need to zero it
    res = Matrix{Float64}(K, N)
    @fastmath @inbounds for n in 1:N
        denominator = Float64(0.0)
        # Use one loop to calculate both numerator and denominator
        for k in 1:K
            res[k,n] = exp(- vecnorm(𝐗[:,n] - 𝐕[:,k]) )
            denominator += res[k,n]
        end
        res[:,n] = res[:,n] .* inv(denominator)
    end
    res
end

# Parallel version
#function softmax_euclidean_par{T<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{T})
function softmax_euclidean_par(𝐗::Matrix{Float64}, 𝐕::Matrix{Float64})
    #nprocs()==CPU_CORES || addprocs(CPU_CORES-1)
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    # Preallocate result matrix, no need to zero it
    res = SharedArray(Float64, (K, N))
    @fastmath @sync @parallel for n in 1:N
        for k in 1:K
            res[k,n] = exp(- vecnorm(𝐗[:,n] - 𝐕[:,k]) )
        end
        res[:,n] = res[:,n] .* inv(sum(res[:,n]))
    end
    res
end

@everywhere function softmax_alpha(𝐗::SharedArray{Float64,2}, 𝐕::SharedArray{Float64,2}, 𝛂::SharedArray{Float64,1})
    return softmax_alpha(sdata(𝐗), sdata(𝐕), sdata(𝛂))
end

# Version that accepts alphas, uses the distance function defined in the paper as Eq (12)
@everywhere function softmax_alpha(𝐗::Matrix{Float64}, 𝐕::Matrix{Float64}, 𝛂::Vector{Float64})
    localD = length(𝛂)
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    # Preallocate result matrix, no need to zero it
    res = Matrix{Float64}(K, N)
    @fastmath @inbounds @simd for n in 1:N
        for k in 1:K
            dm::Float64 = zero(Float64)
            for i in 1:localD # Eq (12)
                dm += 𝛂[i]*((𝐗[i,n]-𝐕[i,k])^2)
            end
            res[k,n] = exp(- dm )
        end
        res[:,n] = res[:,n] .* inv(sum(res[:,n]))
    end
    res
    # Slower alternatives for distance calculation, replacing the 1:localD loop:
    # return 𝛂 ⋅ ((𝐚-𝐛).^2) # Eq (12)
    # return sum(i -> 𝛂[i]*(𝐚[i]-𝐛[i])^2, 1:localD) # Eq (12)
end

@everywhere function softmax_alpha_s(𝐗::SharedArray{Float64,2}, 𝐕::SharedArray{Float64,2}, 𝛂::SharedArray{Float64,1})
    return softmax_alpha_s(sdata(𝐗), sdata(𝐕), sdata(𝛂))
end

# Version that accepts alphas, uses the distance function defined in the paper as Eq (12)
@everywhere function softmax_alpha_s(𝐗::Matrix{Float64}, 𝐕::Matrix{Float64}, 𝛂::Vector{Float64})
    localD = length(𝛂)
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    # Preallocate result matrix as SharedArray, no need to zero it
    res = SharedArray(Float64, (K, N))
    @fastmath @inbounds @simd for n in 1:N
        for k in 1:K
            dm::Float64 = zero(Float64)
            for i in 1:localD # Eq (12)
                dm += 𝛂[i]*((𝐗[i,n]-𝐕[i,k])^2)
            end
            res[k,n] = exp(- dm )
        end
        res[:,n] = res[:,n] .* inv(sum(res[:,n]))
    end
    res
    # Slower alternatives for distance calculation, replacing the 1:localD loop:
    # return 𝛂 ⋅ ((𝐚-𝐛).^2) # Eq (12)
    # return sum(i -> 𝛂[i]*(𝐚[i]-𝐛[i])^2, 1:localD) # Eq (12)
end

SyntaxError: invalid syntax (<ipython-input-129-940f30bfb79d>, line 9)

In [139]:
@time for i in 1:25
    softmax_alpha(𝐗train, 𝐕, 𝛂⁺);
end

In [ ]:
# For development
Zpre = softmax(Xtrain, V)
Zpre_pos = softmax_euclidean_par(Xtrain_pos, V)
Zpre_neg = softmax_euclidean_par(Xtrain_neg, V);

In [140]:
# For development
𝐙pre = softmax_euclidean_par(𝐗train, 𝐕)
𝐙pre⁺ = softmax_euclidean_par(𝐗⁺train, 𝐕)
𝐙pre⁻ = softmax_euclidean_par(𝐗⁻train, 𝐕);

𝐙preu = sdata(𝐙pre)
𝐙preu⁺ = sdata(𝐙pre⁺)
𝐙preu⁻ = sdata(𝐙pre⁻);

1.342045 seconds (8.08 M allocations: 576.790 MB, 11.21% gc time)


This approach is akin to using a funky [multinomial logistic regression](https://en.wikipedia.org/wiki/Multinomial_logistic_regression) to "predict the prototype" (category) where a data point $\mathbf{x}$ maps to.
Wikipedia:
> These are all statistical classification problems. They all have in common a dependent variable to be predicted that comes from one of a limited set of items which cannot be meaningfully ordered, as well as a set of independent variables (also known as features, explanators, etc.), which are used to predict the dependent variable. Multinomial logit regression is a particular solution to the classification problem that assumes that a linear combination of the observed features and some problem-specific parameters can be used to determine the probability of each particular outcome of the dependent variable. The best values of the parameters for a given problem are usually determined from some training data.


## Parts of the optimization objective

### $L_z$ &mdash; statistical parity
In code, we will denote $L_z$ with function `Lz`.

In [141]:
### "NAIVE" VERSION FOR UNDERSTANDING THE IMPLEMENTATION

function LzNaive{T<:Number}(𝐗⁺::Matrix{T}, 𝐗⁻::Matrix{T}, 𝐕::Matrix{T}, dist::Function)
    # Operate on matrices and take mean from sample dimension N
    meanp = mean( softmax_dist_par(𝐗⁺, 𝐕, dist), 2 ) # Eq (6)
    meann = mean( softmax_dist_par(𝐗⁻, 𝐕, dist), 2 ) # Similarly for M_k^-
    sum(abs(meanp - meann)) # Eq (7), sum is from k=1 to K
end

### VERSIONS TAKING IN THE DATA SET AND PROTOTYPES
# Optionally a distance measure function can be passed as an argument

function Lz{T<:Number}(𝐗⁺::Matrix{T}, 𝐗⁻::Matrix{T}, 𝐕::Matrix{T})
    return Lz(sdata(softmax_euclidean_par(𝐗⁺, 𝐕)), sdata(softmax_euclidean_par(𝐗⁻, 𝐕)))
end

function Lz{T<:Number}(𝐗⁺::Matrix{T}, 𝐗⁻::Matrix{T}, 𝐕::Matrix{T}, dist::Function)
    return Lz(sdata(softmax_dist_par(𝐗⁺, 𝐕, dist)), sdata(softmax_dist_par(𝐗⁻, 𝐕, dist)))
end

# # TODO: use parallel version of softmax_dist_alpha?
# function Lz{T<:Number,U<:Number}(𝐗⁺::Matrix{T}, 𝐗⁻::Matrix{T}, 𝐕::Matrix{T}, dist::Function, 𝛂::Vector{U})
#     return Lz(softmax_dist_alpha(𝐗⁺, 𝐕, dist, 𝛂), softmax_dist_alpha(𝐗⁻, 𝐕, dist, 𝛂))
# end

### VERSION FOR PRECALCULATED 𝐙⁺ and 𝐙⁻
# Note we have only a version for matrices. This is because during performance
# testing I noticed that
#
#   ZZZp = sdata(𝐙shared⁺)
#   ZZZn = sdata(𝐙shared⁻)
#   Lz(ZZZp, ZZZn)
#
# is faster than
#
#   Lz(𝐙shared⁺, 𝐙shared⁻)
#
# So use the Matrix version always and if necessary lift the matrices out
# of the SharedArray with sdata().
#
# TODO: is there way to make a faster parallel version?
#function Lz{T<:Number}(𝐙⁺::SharedArray{T,2}, 𝐙⁻::SharedArray{T,2})
@everywhere function Lz(𝐙⁺::SharedArray{Float64,2}, 𝐙⁻::SharedArray{Float64,2})
    Lz(sdata(𝐙⁺), sdata(𝐙⁻))
end

#function Lz{T<:Number}(𝐙⁺::Matrix{T}, 𝐙⁻::Matrix{T})
@everywhere function Lz(𝐙⁺::Matrix{Float64}, 𝐙⁻::Matrix{Float64})
    # Operate on matrices and take mean from sample dimension N
    meanp = mean( 𝐙⁺, 2 ) # Eq (6)
    meann = mean( 𝐙⁻, 2 ) # Similarly for M_k^-
    sum(abs(meanp - meann)) # Eq (7), sum is from k=1 to K
end

In [143]:
# Test (e.g. somewhere between 0.02 and 0.07, depending on 𝐕 randomization)
LZtrain1 = Lz(𝐗⁺test, 𝐗⁻test, 𝐕)
LZtrain2 = Lz(𝐙pre⁺, 𝐙pre⁻)
LZtrain3 = Lz(𝐙preu⁺, 𝐙preu⁻)
(LZtrain1, LZtrain2, LZtrain3)

(0.05031480867026994,0.051942562238902076,0.051942562238902076)

In [144]:
@time for i in 1:5000
    Lz(𝐙pre⁺, 𝐙pre⁻)
end

### $L_x$ &mdash; information loss
In code, we will denote $L_x$ with function `Lx`.

In [146]:
# Symbols: 𝐗 ⁺ ⁻ ∑ 𝐕 𝐱 𝐲 𝐙 𝐳

Note .* elementwise multiplication of softmax_dist() and V, there is no \cdot in the paper in Eq (9), dot product would return a scalar.

In [147]:
### NAIVE VERSION FOR UNDERSTANDING THE IMPLEMENTATION

function LxNaive{T<:Number,U<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{U}, dist::Function)
    D = size(𝐗, 1)
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    𝐗hat = zeros(Float64, (D,N))
    sum = Float64(0.0)
    for n in 1:N
        𝐳_n = softmax_dist(𝐗[:,n], 𝐕, dist) # prob that x_n maps to protos (Array{13,1})
        for k in 1:K # Eq (9)
            𝐗hat[:,n] = 𝐗hat[:,n] + (𝐳_n[k] * 𝐕[:,k])
        end
        sum += (𝐗[:,n] - 𝐗hat[:,n]) ⋅ (𝐗[:,n] - 𝐗hat[:,n]) # Eq (8)
    end
    return sum, 𝐗hat
end

### VERSIONS TAKING IN THE DATA SET AND PROTOTYPES
# Optionally a distance measure function can be passed as an argument

function Lx{T<:Number,U<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{U})
    return Lx(softmax_euclidean_par(𝐗, 𝐕), 𝐗, 𝐕)
end

function Lx{T<:Number,U<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{U}, dist::Function)
    return Lx(softmax_dist_par(𝐗, 𝐕, dist), 𝐗, 𝐕)
end

### VERSION FOR PRECALCULATED 𝐙

#function Lx{T<:Number}(𝐙::SharedArray{T,2}, 𝐗::Matrix{T}, 𝐕::Matrix{T})
@everywhere function Lx(𝐙::SharedArray{Float64,2}, 𝐗::Matrix{Float64}, 𝐕::Matrix{Float64})
    Lx(sdata(𝐙), 𝐗, 𝐕)
end

#function Lx{T<:Number}(𝐙::Matrix{T}, 𝐗::Matrix{T}, 𝐕::Matrix{T})
@everywhere function Lx(𝐙::Matrix{Float64}, 𝐗::Matrix{Float64}, 𝐕::Matrix{Float64})
    𝐗minus𝐗hat = 𝐗 - (𝐕 * 𝐙) # Eq (8) and (9) combined
    return vecdot(𝐗minus𝐗hat, 𝐗minus𝐗hat) # "simple squared error"
end

In [148]:
# Test (e.g. 423475.8479283692)
LXtrain1 = Lx(𝐗train, 𝐕, d)
LXtrain2 = Lx(𝐙pre, 𝐗train, 𝐕)
LXtrain3 = Lx(𝐙preu, 𝐗train, 𝐕)
(LXtrain1, LXtrain2, LXtrain3)

1.298525 seconds (110.12 k allocations: 7.714 MB, 0.32% gc time)


(438406.07313762454,438406.07313762454,438406.07313762454)

In [149]:
@time for i in 1:50
    Lx(𝐙pre, 𝐗train, 𝐕)
end

### $L_y$ &mdash; prediction accuracy
In code, we will denote $L_y$ with function `Ly`.

Essentially here we are letting the optimization pick both the prototypes (i.e. feature vectors) and their predictions (i.e. labels), and the predictions don't have to be discrete 0 and 1, but can be from the range $[0,1]$ and thus themselves can be viewed as probabilities. E.g. let's say that for prototype $v_k$ it's prediction $w_k = 0.82$, then "there is a 82% chance prototype $\mathbf{v}_k$ gets label 1"

In [150]:
### NAIVE VERSION TO HELP UNDERSTAND THE IMPLEMENTATION

function LyNaive{T1<:Number,T2<:Number,T3<:Number}(
        𝐗::Matrix{T1}, 𝐕::Matrix{T1}, 𝐲::Vector{T2}, 𝐰::Vector{T3}, dist::Function
    )
    D = size(𝐗, 1)
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    𝐲hat = zeros(Float64, N)
    sum = Float64(0.0)
    # Replace 𝐲hat in Eq (10) with Eq (11), then you get this for loop
    for n in 1:N
        𝐙_n = softmax_dist(𝐗[:,n], 𝐕, dist) # Vector of length K
        for k in 1:K
            𝐲hat[n] = 𝐲hat[n] + (𝐙_n[k] * 𝐰[k])
        end
        # The following line could be replaced with
        # if 𝐲[n] == 1
        #    sum -= log(𝐲hat[n])
        # else # 𝐲[n] == 0
        #    sum -= log(1 - 𝐲hat[n])
        # end
        sum += -𝐲[n] * log(𝐲hat[n])  -  (1 - 𝐲[n]) * log(1 - 𝐲hat[n])
    end
    #return sum, 𝐲hat
    return sum
end

### VERSIONS TAKING IN THE DATA SET AND PROTOTYPES
# Optionally a distance measure function can be passed as an argument

function Ly{T1<:Number,T2<:Number,T3<:Number}(
        𝐗::Matrix{T1}, 𝐕::Matrix{T1}, 𝐲::Vector{T2}, 𝐰::Vector{T3}
    )
    # 𝐙 = softmax_euclidean_par(𝐗, 𝐕)
    return Ly(softmax_euclidean_par(𝐗, 𝐕), 𝐲, 𝐰)
end

function Ly{T1<:Number,T2<:Number,T3<:Number}(
        𝐗::Matrix{T1}, 𝐕::Matrix{T1}, 𝐲::Vector{T2}, 𝐰::Vector{T3}, dist::Function
    )
    # 𝐙 = softmax_dist_par(𝐗, 𝐕, dist)
    return Ly(softmax_dist_par(𝐗, 𝐕, dist), 𝐲, 𝐰)
end

### VERSION FOR PRECALCULATED 𝐙

# function Ly{T1<:Number,T2<:Number,T3<:Number}(
#         𝐙::Matrix{T1}, 𝐲::Vector{T2}, 𝐰::Vector{T3}
#     )
#     # Copy to shared memory
#     𝐙shared = convert(SharedArray{T1, 2}, 𝐙)
#     return Ly(𝐙shared, 𝐲, 𝐰)
# end

# Slower:
# function Ly{T1<:Number,T2<:Number,T3<:Number}(
#         𝐙::SharedArray{T1,2}, 𝐲::Vector{T2}, 𝐰::Vector{T3}
#     )
#     N = size(𝐙, 2)
#     # Keep 𝐙 as SharedArray, will be faster than taking sdata() when fed to the following @parallel loop
#     sum = @parallel (+) for n in 1:N # Eq (10)
#         yhat_n = 𝐙[:,n] ⋅ 𝐰 # Eq (11)
#         - 𝐲[n] * log(yhat_n) - (1 - 𝐲[n]) * log(1 - yhat_n)
#     end
#     return sum
# end

#function Ly{T1<:Number,T2<:Number,T3<:Number}(𝐙::SharedArray{T1,2}, 𝐲::Vector{T2}, 𝐰::Vector{T3})
@everywhere function Ly(𝐙::SharedArray{Float64,2}, 𝐲::Vector{Int}, 𝐰::SharedArray{Float64,1})
    Ly(sdata(𝐙), 𝐲, sdata(𝐰))
end

@everywhere function Ly(𝐙::SharedArray{Float64,2}, 𝐲::Vector{Int}, 𝐰::Vector{Float64})
    Ly(sdata(𝐙), 𝐲, 𝐰)
end

#function Ly{T1<:Number,T2<:Number,T3<:Number}(𝐙::Matrix{T1}, 𝐲::Vector{T2}, 𝐰::Vector{T3})
@everywhere function Ly(𝐙::Matrix{Float64}, 𝐲::Vector{Int}, 𝐰::Vector{Float64})
    N = size(𝐙, 2)
    sum = zero(Float64)
    𝐲hat = 𝐰' * 𝐙 # Eq (11)
    𝐲hat = reshape(𝐲hat, length(𝐲hat))
    for n in 1:N # Eq (10)
        sum += - 𝐲[n] * log(𝐲hat[n]) - (1 - 𝐲[n]) * log(1 - 𝐲hat[n])
    end
    return sum
end

# Slower:
# function Ly2{T1<:Number,T2<:Number,T3<:Number}(𝐙::Matrix{T1}, 𝐲::Vector{T2}, 𝐰::Vector{T3})
#     tsup = (𝐰' * 𝐙) # # Eq (10) and (11)
#     tsup = reshape(tsup, length(tsup))
#     return -dot(𝐲, log(tsup)) + -dot(1 - 𝐲, log(1 - tsup))
# end

In [151]:
# Test (e.g. 20572.03833866735)
LYtrain1 = Ly(𝐗train, 𝐕, 𝐲train, 𝐰)
LYtrain2 = Ly(𝐙pre, 𝐲train, 𝐰)
LYtrain3 = Ly(𝐙preu, 𝐲train, 𝐰)
(LYtrain1, LYtrain2, LYtrain3)

1.236009 seconds (450 allocations: 2.402 GB, 26.95% gc time)


(19855.692771914662,19855.692771914662,19855.692771914662)

In [152]:
@time for i in 1:200
    Ly(𝐙pre, 𝐲train, 𝐰)
end

## Optimization objective function

In [161]:
# Overall objective function
objective_euclidean(𝐗, 𝐗⁺, 𝐗⁻, 𝐕, 𝐲, 𝐰, A) = A[:z]*Lz(𝐗⁺, 𝐗⁻, 𝐕) + A[:x]*Lx(𝐗, 𝐕) + A[:y]*Ly(𝐗, 𝐕, 𝐲, 𝐰)
objective_dist(𝐗, 𝐗⁺, 𝐗⁻, 𝐕, 𝐲, 𝐰, A, dist) = A[:z]*Lz(𝐗⁺, 𝐗⁻, 𝐕, dist) + A[:x]*Lx(𝐗, 𝐕, dist) + A[:y]*Ly(𝐗, 𝐕, 𝐲, 𝐰, dist)
function objective_pre(𝐗::Matrix, S::Vector{Bool}, 𝐕::Matrix, 𝐲::Vector, 𝐰::Vector, A::Dict)
    # Calculate 𝐙 and partitions once
    𝐙 = softmax_euclidean_par(𝐗, 𝐕)
    (𝐙⁺, 𝐙⁻) = partition(𝐙, S)
    # Use functions that accept precalculated 𝐙
    return A[:z]*Lz(𝐙⁺, 𝐙⁻) + A[:x]*Lx(𝐙, 𝐗, 𝐕) + A[:y]*Ly(𝐙, 𝐲, 𝐰)
end
function objective_pre_alpha(𝐗::Matrix{Float64}, S::Vector{Bool}, 𝐕::Matrix{Float64}, 𝐲::Vector{Int}, 𝐰::Vector{Float64}, A::Dict{Symbol, Float64}, 𝛂⁺::Vector{Float64}, 𝛂⁻::Vector{Float64})
    # Calculate 𝐙 and partitions
    (𝐗⁺, 𝐗⁻) = partition(𝐗, S)
    (𝐲⁺, 𝐲⁻) = partition(𝐲, S)
    𝐙⁺ = softmax_alpha_s(𝐗⁺, 𝐕, 𝛂⁺)
    𝐙⁻ = softmax_alpha_s(𝐗⁻, 𝐕, 𝛂⁻)
    # Use functions that accept precalculated 𝐙
    return A[:z]*Lz(𝐙⁺, 𝐙⁻) + A[:x]*Lx(𝐙⁺, 𝐗⁺, 𝐕) + A[:x]*Lx(𝐙⁻, 𝐗⁻, 𝐕) + A[:y]*Ly(𝐙⁺, 𝐲⁺, 𝐰) + A[:y]*Ly(𝐙⁻, 𝐲⁻, 𝐰)
end
function objective_pre_alpha_s(𝐗::SharedArray{Float64,2}, S::SharedArray{Bool,1}, 𝐕::Matrix{Float64}, 𝐲::SharedArray{Int,1}, 𝐰::Vector{Float64}, A::Dict{Symbol, Float64}, 𝛂⁺::Vector{Float64}, 𝛂⁻::Vector{Float64})
    # Calculate 𝐙 and partitions
    (𝐗⁺, 𝐗⁻) = partition(𝐗, S)
    (𝐲⁺, 𝐲⁻) = partition(𝐲, S)
    @sync begin
        𝐙⁺ref = @spawn softmax_alpha_s(𝐗⁺, 𝐕, 𝛂⁺)
        𝐙⁻ref = @spawn softmax_alpha_s(𝐗⁻, 𝐕, 𝛂⁻)
    end
    𝐙⁺ = fetch(𝐙⁺ref)
    𝐙⁻ = fetch(𝐙⁻ref)
    @sync begin
        rlz = @spawn Lz(𝐙⁺, 𝐙⁻)
        rlxp = @spawn Lx(𝐙⁺, 𝐗⁺, 𝐕)
        rlxn = @spawn Lx(𝐙⁻, 𝐗⁻, 𝐕)
        rlyp = @spawn Ly(𝐙⁺, 𝐲⁺, 𝐰)
        rlyn = @spawn Ly(𝐙⁻, 𝐲⁻, 𝐰)
    end
    # Use functions that accept precalculated 𝐙
    #return A[:z]*Lz(𝐙⁺, 𝐙⁻) + A[:x]*Lx(𝐙⁺, 𝐗⁺, 𝐕) + A[:x]*Lx(𝐙⁻, 𝐗⁻, 𝐕) + A[:y]*Ly(𝐙⁺, 𝐲⁺, 𝐰) + A[:y]*Ly(𝐙⁻, 𝐲⁻, 𝐰)
    return A[:z]*fetch(rlz) + A[:x]*fetch(rlxp) + A[:x]*fetch(rlxn) + A[:y]*fetch(rlyp) + A[:y]*fetch(rlyn)
end

objective_pre_alpha_s (generic function with 2 methods)

In [154]:
objective_euclidean(𝐗train, 𝐗⁺train, 𝐗⁻train, 𝐕, 𝐲train, 𝐰, A)

0.328048 seconds (1.66 k allocations: 49.763 MB, 3.73% gc time)


2081.352446744131

In [155]:
objective_pre(𝐗train, S_𝐗train, 𝐕, 𝐲train, 𝐰, A)

2081.352446744131

In [156]:
objective_pre_alpha(𝐗train, S_𝐗train, 𝐕, 𝐲train, 𝐰, A, 𝛂⁺, 𝛂⁻)

2951.8650964392987

In [157]:
𝐗train_s = convert(SharedArray{Float64, 2}, 𝐗train)
S_𝐗train_s = convert(SharedArray{Bool, 1}, S_𝐗train)
𝐲train_s = convert(SharedArray{Int, 1}, 𝐲train);

In [162]:
objective_pre_alpha_s(𝐗train_s, S_𝐗train_s, 𝐕, 𝐲train_s, 𝐰, A, 𝛂⁺, 𝛂⁻)

2951.8650964392987

In [33]:
objective_euclidean(𝐗test, 𝐗⁺test, 𝐗⁻test, 𝐕, 𝐲test, 𝐰, A)

1220.3472494405382

In [167]:
# @time for i in 1:10
#     objective_euclidean(𝐗train, 𝐗⁺train, 𝐗⁻train, 𝐕, 𝐲train, 𝐰, A)
# end
# @time for i in 1:10
#     objective_pre(𝐗train, S_𝐗train, 𝐕, 𝐲train, 𝐰, A)
# end
# @time for i in 1:10
#     objective_pre_alpha(𝐗train, S_𝐗train, 𝐕, 𝐲train, 𝐰, A, 𝛂⁺, 𝛂⁻)
# end
# @time for i in 1:10
#     objective_pre_alpha_s(𝐗train_s, S_𝐗train_s, 𝐕, 𝐲train_s, 𝐰, A, 𝛂⁺, 𝛂⁻)
# end
#   5.755369 seconds (831.70 k allocations: 558.355 MB, 1.60% gc time)
#   2.124663 seconds (210.88 k allocations: 557.520 MB, 3.58% gc time)
#   1.753765 seconds (13.20 M allocations: 1.100 GB, 35.45% gc time)
#   1.303245 seconds (84.00 k allocations: 283.905 MB, 2.54% gc time)

  5.755369 seconds (831.70 k allocations: 558.355 MB, 1.60% gc time)
  2.124663 seconds (210.88 k allocations: 557.520 MB, 3.58% gc time)
  1.753765 seconds (13.20 M allocations: 1.100 GB, 35.45% gc time)
  1.303245 seconds (84.00 k allocations: 283.905 MB, 2.54% gc time)


In [168]:
# @time for i in 1:100
#     objective_pre_alpha(𝐗train, S_𝐗train, 𝐕, 𝐲train, 𝐰, A, 𝛂⁺, 𝛂⁻)
# end
# @time for i in 1:100
#     objective_pre_alpha_s(𝐗train_s, S_𝐗train_s, 𝐕, 𝐲train_s, 𝐰, A, 𝛂⁺, 𝛂⁻)
# end
#  17.722520 seconds (132.00 M allocations: 10.997 GB, 34.11% gc time)
#  13.390848 seconds (858.29 k allocations: 2.773 GB, 2.72% gc time)

 17.722520 seconds (132.00 M allocations: 10.997 GB, 34.11% gc time)
 13.390848 seconds (858.29 k allocations: 2.773 GB, 2.72% gc time)


In [35]:
function fun_to_profile(n::Int)
   for i in 1:n
        objective_pre_alpha(𝐗train, S_𝐗train, 𝐕, 𝐲train, 𝐰, A, 𝛂⁺, 𝛂⁻)
    end
end

# Profiling
profiling_logfile = "profile.bin"
function benchmark()
    # Any setup code goes here.

    # Run once, to force compilation.
    println("======================= First run:")
    srand(666)
    @time fun_to_profile(1)

    # Run a second time, with profiling.
    println("\n\n======================= Second run:")
    srand(666)
    Profile.init(delay=0.01)
    Profile.clear()
    Profile.clear_malloc_data()
    @profile @time fun_to_profile(50)
    
    Profile.print()

#     # Write profile results to profile.bin.
#     r = Profile.retrieve()
#     f = open(profiling_logfile, "w")
#     serialize(f, r)
#     close(f)
end

# function show_profiling()
#     f = open(profiling_logfile)
#     r = deserialize(f);
#     ProfileView.view(r[1], lidict=r[2])
# end

benchmark (generic function with 1 method)

In [36]:
# benchmark()

In [37]:
#show_profiling()

## Test optimization run

In [169]:
count = 0 # keep track of # function evaluations
#function obj_func(x::Vector, grad::Vector)
function obj_func(x::Vector{Float64})
#function obj_func(x::Vector)
#     return sum(x)
    # Increase count
    global count
    count::Int += 1
    
    # Print progress
    if count % 250 == 0
        @printf("round %d, ", count)
    end
    
    # Get 𝐕, 𝐰, 𝛂⁺, 𝛂⁻
    loc𝐕 = reshape(x[1:length(𝐕)], size(𝐕)) # Uses the global 𝐕 for size
    cursor = length(𝐕)
    loc𝛂⁺ = x[cursor+1:cursor+length(𝛂⁺)]
    cursor += length(𝛂⁺)
    loc𝛂⁻ = x[cursor+1:cursor+length(𝛂⁻)]
    cursor += length(𝛂⁻)
    loc𝐰 = x[cursor+1:cursor+length(𝐰)]
    #return size(loc𝐕), size(loc𝛂⁺), size(loc𝛂⁻), size(loc𝐰)
#     return objective_pre_alpha(𝐗train, S_𝐗train, loc𝐕, 𝐲train, loc𝐰, A, loc𝛂⁺, loc𝛂⁻)
    return objective_pre_alpha_s(𝐗train_s, S_𝐗train_s, loc𝐕, 𝐲train_s, loc𝐰, A, loc𝛂⁺, loc𝛂⁻)
#     return objective_pre_alpha(
#         𝐗train::Matrix{Float64},
#         S_𝐗train::Vector{Bool},
#         loc𝐕::Vector{Float64},
#         𝐲train::Vector{Int},
#         loc𝐰::Vector{Float64},
#         A::Dict{Symbol, Float64},
#         loc𝛂⁺::Vector{Float64},
#         loc𝛂⁻::Vector{Float64}
#     )
#     if length(grad) > 0:
#         ...set grad to gradient, in-place...
#     return ...value of f(x)...
end

obj_func (generic function with 1 method)

In [170]:
?reshape

search: 

```
reshape(A, dims)
```

Create an array with the same data as the given array, but with different dimensions. An implementation for a particular type of array may choose whether the data is copied or shared.


reshape promote_shape



In [171]:
size(𝐕),size(𝐰'),size(𝛂⁺),size(𝛂⁻)

((99,13),(1,13),(99,),(99,))

In [172]:
optVarInit = vcat(reshape(𝐕, length(𝐕)), reshape(𝛂⁺, length(𝛂⁺)), reshape(𝛂⁻, length(𝛂⁻)), reshape(𝐰, length(𝐰)));

In [173]:
zeros(Int,1)

1-element Array{Int64,1}:
 0

In [174]:
bounds_lower = vcat(fill(-Inf, length(𝐕)), zeros(length(𝛂⁺) + length(𝛂⁻) + length(𝐰)));
bounds_upper = vcat(fill(+Inf, length(𝐕)), ones(length(𝛂⁺) + length(𝛂⁻) + length(𝐰)));
length(optVarInit), length(bounds_lower), length(bounds_upper)
bounds = collect(zip(bounds_lower, bounds_upper));

In [175]:
# bounds = collect(zip(zeros(3), ones(3)))
bounds[1:3]

3-element Array{Tuple{Float64,Float64},1}:
 (-Inf,Inf)
 (-Inf,Inf)
 (-Inf,Inf)

In [177]:
count::Int = 0
res = scipyopt.fmin_l_bfgs_b(
    obj_func,
    x0=optVarInit,
    #x0=optVarInit[1:3],
    epsilon=1e-5,
#     args=(
#         training_sensitive, training_nonsensitive, 
#         ytrain_sensitive, ytrain_nonsensitive, k, 1e-4,
#         0.1, 1000, 0
#     ),
    bounds=bounds,
    #bounds=bounds,
    approx_grad=true,
    maxfun=500,
    maxiter=500
)

round 250, round 500, round 750, round 1000, round 1250, 

LoadError: LoadError: InterruptException:
while loading In[177], in expression starting on line 2

In [ ]:
# res[1]

In [ ]:
# count::Int = 0
# # # Call L-BFGS
# # res = optimize(obj_func,
# #     optVarInit,
# #     method = :l_bfgs,
# #     xtol = 1e-4,
# #     grtol = 1e-12,
# #     iterations = 1000,
# #     store_trace = true,
# #     show_trace = false)

# # TODO: Can we use upper and lower limits for 𝐕? Will it speed up the optimizer?
# #       The prototypes need to lie inside the smallest hypercube that contains all original datapoints, yes?

# d1 = DifferentiableFunction(obj_func)
# # # Note that d1 above will use central finite differencing to approximate the gradient.

# res = fminbox(d1,
#     optVarInit,
#     bounds_lower,
#     bounds_upper,
#     xtol = 1e-4,
#     grtol = 1e-12,
#     iterations = 100,
#     store_trace = true,
#     show_trace = false)
# # In tutorial: @elapsed fminbox(d4, x0, l, u)

In [ ]:
# #opt = Opt(:LD_MMA, 2)
# opt = Opt(:LN_SBPLX, length(optVarInit)) # Use a derivative-free optimization algorithm (instead of L-BFGS)

# # Lower and upper bounds for alphas and 𝐰
# lower_bounds!(opt, bounds_lower)
# upper_bounds!(opt, bounds_upper)

# # Tolerance
# xtol_rel!(opt, 1e-4)

# # Stop when the number of function evaluations exceeds the second argument.
# #(0 or negative for no limit.)
# maxeval!(opt, 200)

# # Stop when the optimization time (in seconds) exceeds the second argument.
# #(0 or negative for no limit.)
# maxtime!(opt, 60*2)

# # Minimize
# min_objective!(opt, obj_func)

# (minf,minx,ret) = NLopt.optimize(opt, optVarInit)
# println("got $minf at $minx after $count iterations (returned $ret)")

# Optimization, running the algorithm
Hyperparameters for the objective function.
In the paper they use grid search to find the parameters. The sets defined here are the same as in the paper.

In [ ]:
# Sets of hyperparameters as in paper, for grid search
gridA = Dict(:z => Set([0.1, 0.5, 1.0, 5.0, 10.0]), :x => Set([0, 0.01]), :y => Set([0.1, 0.5, 1.0, 5.0, 10.0]))
# An example of selected hyperparameters, for development
A = Dict(:z => 0.01, :x => 0.5, :y => 1.0)

# TODO
- Overall process with pictures
- Some analysis e.g. check the alphas if they give the same result
- Return

# Problems/Cons/Notes:
- Nothing said in the paper about choosing the number of prototypes $K$
- Let's say that there is a column/feature "Religion" in the dataset.
- Now this paper says we can only say that "Is a member of protected group" or "Is not a member of protected group".
- You have to decide what is the "protected" group, and what is the "normal/non protected" group. You have to decide based on some external criteria who are discriminated against and who are not.
- Let's say we have a dataset with a feature "Religion" and we have 5 different religions represented.
- Now we have to choose which ones are protected and which ones are not.
- The problem of course is that some might be in general discriminated against more than others. There is not necessary even split between the different groups that are discriminated against.

- What we would like to say is that "Religion" is a sensitive feature, and we should not infer _anything_ from it, regardless what it is.

- Does running the algo multiple times help, changing the binary classification each time? Can we extend it so that $S \in {1,...,C}$ where $C$ is the number of categories in the sensitive column.
  - We can extend, just split $L_z$ to multiple cases and the optimization is done to all of them. There will be $c = \frac{(C-1)C}{2} \approx O(C^2)$ pairs. Whether this is computationally still feasible is another question. In the objective function $L_z$ is replaced by $A_{z_1} \cdot L_{z_1} + A_{z_2} \cdot L_{z_2} + \dots + A_{z_c} \cdot L_{z_c}$.

- On the current case where $S \in \left\{0,1\right\}$ once we have set for which rows $S=1$ and $S=0$, we can flip them around without changing anything. This is because we are using statistical parity. This means that from the algorithm's perspective saying that group0 is non-protected and groups 1..4 are protected is the same as saying group1 is protected and other non-protected.

## Problems with Julia / libraries
- Optim.jl and NLOpt.jl don't work for some reason on my Mac. Need to check if the problem persists on Linux. This caused maybe 20 hours of extra work, until I figured out I can use PyCall to call SciPy's L-BFGS implementation.
- I've spent hours optimizing the code but it is still too slow. Should go through Julia's performance manual again.